In [ ]:
# Import some tools to build dates 
from datetime import date

import pandas as pd

import matplotlib.pyplot as plt

# Import the function to get connect to the db
from snowexsql.db import get_db

from snowexsql.data import LayerData, PointData, ImageData, SiteData

# Import the function to investigate a table
from snowexsql.db import get_table_attributes
# Import a useful function to format that data into a dataframe
from snowexsql.conversions import query_to_geopandas

In [ ]:
# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@52.32.183.144/snowex'

# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)

In [ ]:
# Use the function to see what columns are available to use. 
db_columns = get_table_attributes(LayerData)

# Print out the results nicely
print("These are the available columns in the table:\n \n* {}\n".format('\n* '.join(db_columns)))

In [ ]:
# Pick a dataset
dataset = 'temperature'

# Pick a date
collection_date = date(2020, 2, 8)

qry = session.query(LayerData)

qry = qry.filter(LayerData.type == 'temperature')

# Filter by a date
#qry = qry.filter(LayerData.date == collection_date)

# Limit it to a couple thousand 
qry = qry.limit(2000)

# Execute the query and convert to geopandas in one handy function
df = query_to_geopandas(qry, engine)

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

Plot

In [ ]:
## Get the Matplotlib Axes object from the dataframe object, color the points by snow depth value
#ax = df.plot(column='value', legend=False, cmap='RdBu')
#
## Use non-scientific notation for x and y ticks
#ax.ticklabel_format(style='plain', useOffset=False)
#
## Set the various plots x/y labels and title.
#ax.set_title(f'{len(df.index)} {dataset.title()}s collected on {collection_date.strftime("%Y-%m-%d")}')
#ax.set_xlabel('Easting [m]')
#ax.set_ylabel('Northing [m]')
#
## Close the session to avoid hanging transactions
#session.close()

In [ ]:
df_pd = pd.DataFrame(df)

In [ ]:
df_pd.columns

In [ ]:
# make sure that our snow temperature values are a number
df_pd['value'] = df_pd.value.astype(float)
# and that site ids are strings
df_pd['site_id'] = df_pd.site_id.astype(str)

Vegetation classes

    1-3 = treeless (0% tree cover)

    4-6 = sparse (1-30% tree cover)

    7-9 = dense (31-100% tree cover)

Snow Depth classes

    1, 4, 7 = shallow (<90cm)

    2, 5, 8 = medium (90-122cm)

    3, 6, 9 = deep (>122cm)

Flight Lines

    ‘C’ = Crossline

    ‘N’ = North

    ‘S’ = South


In [ ]:
vegetation_classes = {'0':'na', 'T':'na', 'G':'na', 'F':'na',
                      '1':'treeless', '2':'treeless', '3':'treeless', 
                      '4':'sparse', '5':'sparse', '6':'sparse', 
                      '7':'dense', '8':'dense', '9':'dense'}
snow_classes = {'0':'na', 'T':'na', 'G':'na', 'F':'na',
                '1':'shallow', '2':'medium', '3':'deep', 
                '4':'shallow', '5':'medium', '6':'deep', 
                '7':'shallow', '8':'medium', '9':'deep'}
id_string_prefix = [id_string[0:2] for id_string in df_pd['site_id']]
df_pd['veg_class'] = [ vegetation_classes[id_string[0]] for id_string in df_pd['site_id']]
df_pd['snow_class'] = [ snow_classes[id_string[0]] for id_string in df_pd['site_id']]

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))

df_pd.groupby('snow_class').plot(x='value',y='depth', ax=ax, linestyle='-', marker='o');

ax.set_title('Snow Pit Temperature Profiles')
ax.set_xlabel('Snow temperature ($\degree C$)')
ax.set_xlim((-30,0))
ax.set_ylabel('Snow layer depth (cm)')
ax.set_ylim((0,120))

In [ ]:
for i, this_date in enumerate(df_pd.date.unique()):
    
    fig, ax = plt.subplots(figsize=(5,5))
    
    df_pd.where(df.date==this_date).groupby('site_id').plot(x='value',y='depth', ax=ax, linestyle='-', marker='o', label='site_id');
    
    ax.set_title('Snow Pit Temperature Profiles\n{}'.format(this_date))
    ax.set_xlabel('Snow temperature ($\degree C$)')
    ax.set_xlim((-30,0))
    ax.set_ylabel('Snow layer depth (cm)')
    ax.set_ylim((0,120))